In [1]:
import re, os, sys
import pandas as pd
import numpy as np
import py_stringmatching as sm

In [2]:
path = '/Users/galvanize/infosys/cip/data/nc_data'
filename0, filename1 = 'nc0.csv', 'nc1.csv'
filepath0, filepath1 = os.path.join(path, filename0), os.path.join(path, filename1)

In [3]:
df0 = pd.read_csv(filepath0, dtype={'recid':np.int64, 'givenname':str, 'surname':str, 'suburb':str, 'postcode':str})
df1 = pd.read_csv(filepath1, dtype={'recid':np.int64, 'givenname':str, 'surname':str, 'suburb':str, 'postcode':str})
ROW_BOUNDARY = len(df0) #separation btw first and second files
df = pd.concat([df0, df1]).reset_index(drop=True)

answers = df['recid']
df = df.drop(['recid'], axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000000 entries, 0 to 1999999
Data columns (total 4 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   givenname  object
 1   surname    object
 2   suburb     object
 3   postcode   object
dtypes: object(4)
memory usage: 61.0+ MB


In [4]:
for col in df.columns:
    df.loc[:,col] = df[col].apply(lambda x: str(x)).str.replace('[^A-Za-z0-9]+', '', regex=True)

In [5]:
tok = sm.QgramTokenizer(qval=3, return_set=True, padding=False)

In [6]:
for col in df.columns:
    df.loc[:,f'{col}_qgrams'] = df[col].apply(lambda x: sorted(tok.tokenize(x)))
df.head()

,givenname,surname,suburb,postcode,givenname_qgrams,surname_qgrams,suburb_qgrams,postcode_qgrams
0,kadelyn,gragnani,waxhaw,2873,"[ade, del, ely, kad, lyn]","[agn, ani, gna, gra, nan, rag]","[axh, haw, wax, xha]","[287, 873]"
1,ronel,carter,washington,2788g,"[nel, one, ron]","[art, car, rte, ter]","[ash, gto, hin, ing, ngt, shi, ton, was]","[278, 788, 88g]"
2,jason,roehrig,millersceeek,286s1,"[aso, jas, son]","[ehr, hri, oeh, rig, roe]","[cee, eee, eek, ers, ill, ler, lle, mil, rsc, ...","[286, 6s1, 86s]"
3,antrea,mure,greensboro,27410,"[ant, ntr, rea, tre]","[mur, ure]","[bor, een, ens, gre, nsb, oro, ree, sbo]","[274, 410, 741]"
4,caedar,oliver,jacksonville,28542,"[aed, cae, dar, eda]","[ive, liv, oli, ver]","[ack, cks, ill, jac, kso, lle, nvi, onv, son, ...","[285, 542, 854]"


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000000 entries, 0 to 1999999
Data columns (total 8 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   givenname         object
 1   surname           object
 2   suburb            object
 3   postcode          object
 4   givenname_qgrams  object
 5   surname_qgrams    object
 6   suburb_qgrams     object
 7   postcode_qgrams   object
dtypes: object(8)
memory usage: 122.1+ MB


In [8]:
all_suburb_qgrams = set().union(*df['suburb_qgrams'].values)
all_givenname_qgrams = set().union(*df['givenname_qgrams'].values)
all_surname_qgrams = set().union(*df['surname_qgrams'].values)
all_postcode_qgrams = set().union(*df['postcode_qgrams'].values)
len(all_suburb_qgrams), len(all_givenname_qgrams), len(all_surname_qgrams), len(all_postcode_qgrams)

(12536, 13470, 16608, 2199)

In [9]:
from collections import defaultdict
from itertools import combinations
from more_itertools import partition

In [ ]:
postcode_dict = dict(zip(df.index, df.postcode_qgrams))
inverted_postcode_dict = defaultdict(list)
for key, values in postcode_dict.items():
    for value in values:
        inverted_postcode_dict[value].append(key)

rows_to_compare_postcode = set()
for key in inverted_postcode_dict.keys():
    _, combos = partition(lambda t: t[0] < ROW_BOUNDARY and t[1] >= ROW_BOUNDARY,
                        combinations(inverted_postcode_dict[key], 2))
    rows_to_compare_postcode.update(set(combos))

len(rows_to_compare_postcode)

In [ ]:
len(rows_to_compare_postcode)

In [ ]:
suburb_dict = dict(zip(df.index, df.suburb_qgrams))
inverted_suburb_dict = defaultdict(list)
for key, values in suburb_dict.items():
    for value in values:
        inverted_suburb_dict[value].append(key)

rows_to_compare_suburb = set()
for key in inverted_suburb_dict.keys():
    _, combos = partition(lambda t: t[0] < ROW_BOUNDARY and t[1] >= ROW_BOUNDARY,
                        combinations(inverted_suburb_dict[key], 2))
    rows_to_compare_suburb.update(set(combos))
len(rows_to_compare_suburb)

In [ ]:
surname_dict = dict(zip(df.index, df.surname_qgrams))
inverted_surname_dict = defaultdict(list)
for key, values in surname_dict.items():
    for value in values:
        inverted_surname_dict[value].append(key)

rows_to_compare_surname = set()
for key in inverted_surname_dict.keys():
    _, combos = partition(lambda t: t[0] < ROW_BOUNDARY and t[1] >= ROW_BOUNDARY,
                        combinations(inverted_surname_dict[key], 2))
    rows_to_compare_surname.update(set(combos))
len(rows_to_compare_surname)

In [ ]:
givenname_dict = dict(zip(df.index, df.givenname_qgrams))
inverted_givenname_dict = defaultdict(list)
for key, values in givenname_dict.items():
    for value in values:
        inverted_givenname_dict[value].append(key)

rows_to_compare_givenname = set()
for key in inverted_givenname_dict.keys():
    _, combos = partition(lambda t: t[0] < ROW_BOUNDARY and t[1] >= ROW_BOUNDARY,
                        combinations(inverted_givenname_dict[key], 2))
    rows_to_compare_givenname.update(set(combos))
len(rows_to_compare_givenname)

In [ ]:
final = (set().union(rows_to_compare_givenname,
                     rows_to_compare_surname,
                     rows_to_compare_suburb,
                     rows_to_compare_postcode
                    )
        )

len(final)

In [ ]:
ROW_BOUNDARY**2

In [ ]:
len(final)/ROW_BOUNDARY**2